In [1]:
import ast
from bs4 import BeautifulSoup
import bs4
from urllib.request import urlopen, Request
from urllib.error import HTTPError
import mechanicalsoup
import time
import os
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image
from selenium.common.exceptions import NoSuchElementException
import io
import requests
import numpy as np
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
import re


## Working with dynamic webpages

Some of the websites we visit have popup and adverts. We can load an extension (uBlock Origin) to prevent these from affecting data collection.

In [ ]:
# extension Ublock origin
epath = '/Users/marissakhoury/Library/Application Support/Google/Chrome/Default/Extensions/cjpalhdlnbpafiamejdnhcphjbkeiagm/1.31.2_0'
chrome_path = '/Users/marissakhoury/Desktop/webscrape/drivers/chromedriver'

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('load-extension=' + epath)
chrome_options.add_argument('chrome')

driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)
driver.create_options()
driver.get("https://www.misshaus.com/default/super-aqua-snail-cream.html")
driver.close()

# Data Cleaning

In [3]:
# sephora data
sephora_raw = pd.read_csv('full_data.csv')
sephora_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1196 entries, 0 to 1195
Data columns (total 14 columns):
Unnamed: 0          1196 non-null int64
Label               963 non-null object
URL                 1196 non-null object
brand               1188 non-null object
description         1063 non-null object
full_description    225 non-null object
ingredients         1188 non-null object
name                1188 non-null object
num_reviews         1188 non-null float64
price               1188 non-null float64
prod_size           1109 non-null object
rating              1188 non-null float64
skin_type           1128 non-null object
i                   1196 non-null int64
dtypes: float64(3), int64(2), object(9)
memory usage: 130.9+ KB


In [4]:
sephora_raw.head()

,Unnamed: 0,Label,URL,brand,description,full_description,ingredients,name,num_reviews,price,prod_size,rating,skin_type,i
0,0,moisturizing-cream-oils-mists,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,"['Glacial Glycoprotein', 'Squalene', 'Aqua/Wat...",Ultra Facial Cream,1132.0,32.0,1.7,4.3,"['Dry', 'Normal', 'Combination']",0
1,1,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,"['Centella Asiatica Complex', 'Herbs Complex',...",Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.7 fl.,4.1,"['Normal', 'Dry', 'Combination', 'Oily']",1
2,2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,"['Signal Peptide Complex (Growth Factors)', 'P...",Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,"['Normal', 'Dry', 'Combination', 'Oily']",2
3,3,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,"['Algae (Seaweed) Extract', 'Mineral Oil', 'Pe...",Crème de la Mer Moisturizer,695.0,190.0,1,3.9,['Dry'],3
4,4,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,"['Aqua/Water/Eau', 'Saccharomyces/Camellia Sin...",The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.7,4.2,"['Normal', 'Dry']",4


## Cleaning the ingredients

We've parsed the ingredients from the website. In the raw data, the ingredients column contains strings, string representations of list. For example

```
"['Glacial Glycoprotein', 'Squalene', 'Aqua/Water', 'Glycerin', 'Cyclohexasiloxane', 'Squalane', 'Bis-PEG-18 Methyl Ether Dimethyl Silane', 'Sucrose Stearate', 'Stearyl Alcohol', 'PEG-8 Stearate', 'Myristyl Myristate', 'Pentaerythrityl Tetraethlhexanoate', 'Prunus Armeniaca Kernal Oil/Apricot Kernal Oil', 'Phenoxyethanol', 'Persea Gratissima Oil/Avocado Oil', 'Glyceryl Stearate', 'Cetl Alcohol', 'Oryza Sativa Bran Oil/Rice Bran Oil', 'Olea Europaea Fruit Oil/G4Olive Fruit Oil', 'Chlorphenesin', 'Stearic Acid', 'Palmitic Acid', 'Disodium EDTA', 'Acrylates/C10-30 Alkyl Acrylate Crosspolymer', 'Carbomer', 'Prunus Amygdalus Dulcis Oil/Sweet Almond Oil', 'Xanthan Gum', 'Ethylhexylglycerin', 'Sodium Hydroxide', 'Tocopherol', 'Glycine Soja Oil/Soybean Oil', 'Pseudoalteromonas Ferment Extract', 'Myristic acid', 'Hydroxypalmitol Sphinganine', 'BHT', 'Salicylic Acid', 'Citric Acid.']"
```

There is a consistency issue with the ingredients listed on the product description. There are cases where certain ingredients carry two names, a description, or even punctuation/grammar. 

In [5]:
# let's grab the ones with ingredients
products_with_ingredients = sephora_raw[sephora_raw.ingredients.notnull()]
products_with_ingredients.head()

,Unnamed: 0,Label,URL,brand,description,full_description,ingredients,name,num_reviews,price,prod_size,rating,skin_type,i
0,0,moisturizing-cream-oils-mists,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,"['Glacial Glycoprotein', 'Squalene', 'Aqua/Wat...",Ultra Facial Cream,1132.0,32.0,1.7,4.3,"['Dry', 'Normal', 'Combination']",0
1,1,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,"['Centella Asiatica Complex', 'Herbs Complex',...",Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.7 fl.,4.1,"['Normal', 'Dry', 'Combination', 'Oily']",1
2,2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,"['Signal Peptide Complex (Growth Factors)', 'P...",Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,"['Normal', 'Dry', 'Combination', 'Oily']",2
3,3,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,"['Algae (Seaweed) Extract', 'Mineral Oil', 'Pe...",Crème de la Mer Moisturizer,695.0,190.0,1,3.9,['Dry'],3
4,4,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,"['Aqua/Water/Eau', 'Saccharomyces/Camellia Sin...",The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.7,4.2,"['Normal', 'Dry']",4


### Parsing Sephora Description

I noticed widely used two formats on Sephora's product page. We'll write a method to parse both of these ways.


Skin Types: 

[Normal, Oily, Combination, Dry, Sensitive]

In [6]:
skin_type = ['✔ Normal', '✔ Oily', '✔ Combination', '✔ Dry', '✔ Sensitive']
word_pattern = r'(\w+)'
skin_type = list(map(lambda x: re.search(word_pattern, x).group(1), skin_type))

In [7]:
def clean_description(text):
    lines = text.split('\n')
    if '✔' in text:
        # listed
        skin_type = list(filter(lambda x: '✔' in x, lines))
        word_pattern = r'(\w+)'
        skin_type = list(map(lambda x: re.search(word_pattern, x).group(1), skin_type)) # parse word from text
        
        # description comes after 'What it is:'
        description_start_index = lines.index('What it is:')
        description = lines[description_start_index+1]
        return skin_type, description
        
    else:
        # raw text
        # description comes first
        description = lines[0]
        pre_text = 'What it is: '
        description = description[description.index(pre_text) + len(pre_text):]
        pre_text = 'Skin Type: '
        skin_type_str = lines[1]
        skin_type_str = skin_type_str[skin_type_str.index(pre_text) + len(pre_text):]
        skin_type = list(filter(lambda x: x != 'and', re.findall(r'\w+', skin_type_str)))
        return skin_type, description
    
def clean_ingredients(ingredients):
    if ingredients is np.nan:
        return []
    il = ast.literal_eval(ingredients)
    parsed_ingredients = []
    for ing in il:
        # clean single ingredient
        ingredient = ing.strip(' .*[]')
        words = set(ingredient.split())
        if 'not' in words or\
            'are' in words or\
            'at' in words or\
            'and' in words or\
            'or' in words or\
            'the' in words or\
            len(words) > 5:
            continue

        parsed_ingredients.append(ingredient)
    return parsed_ingredients


In [ ]:
def parse_skin_text(s):
    try:
        return list(filter(lambda x: x != 'and', re.findall(r'\w+', s)))
    except:
        return None

products_with_ingredients['skin_type'] = products_with_ingredients['skin_type'].apply(parse_skin_text)

In [10]:
products_with_ingredients.head()

,Unnamed: 0,Label,URL,brand,description,full_description,ingredients,name,num_reviews,price,prod_size,rating,skin_type,i
0,0,moisturizing-cream-oils-mists,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,"['Glacial Glycoprotein', 'Squalene', 'Aqua/Wat...",Ultra Facial Cream,1132.0,32.0,1.7,4.3,"[Dry, Normal, Combination]",0
1,1,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,"['Centella Asiatica Complex', 'Herbs Complex',...",Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.7 fl.,4.1,"[Normal, Dry, Combination, Oily]",1
2,2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,"['Signal Peptide Complex (Growth Factors)', 'P...",Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,"[Normal, Dry, Combination, Oily]",2
3,3,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,"['Algae (Seaweed) Extract', 'Mineral Oil', 'Pe...",Crème de la Mer Moisturizer,695.0,190.0,1,3.9,[Dry],3
4,4,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,"['Aqua/Water/Eau', 'Saccharomyces/Camellia Sin...",The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.7,4.2,"[Normal, Dry]",4


In [ ]:
for x in products_with_ingredients.index:
    try:
        skin_type, description = clean_description(products_with_ingredients['full_description'][x])
        products_with_ingredients['skin_type'][x] = skin_type
        products_with_ingredients['description'][x] = description
    except:
        continue
    

In [34]:
fdata = pd.read_csv('v1_data_clean.csv')

def f(x):
    try:
        return ast.literal_eval(x)
    except:
        return []

# eval string of list into list
fdata.skin_type = fdata.skin_type.apply(f)
fdata.ingredients = fdata.ingredients.apply(f)
fdata.head()

,Unnamed: 0,Label,URL,brand,description,full_description,ingredients,name,num_reviews,price,prod_size,rating,skin_type
0,0,moisturizing-cream-oils-mists,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,"[Glacial Glycoprotein, Squalene, Aqua/Water, G...",Ultra Facial Cream,1132.0,32.0,1.7,4.3,"[Dry, Normal, Combination]"
1,1,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,"[Centella Asiatica Complex, Herbs Complex, Wat...",Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.7 fl.,4.1,"[Normal, Dry, Combination, Oily]"
2,2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,"[Signal Peptide Complex (Growth Factors), Pygm...",Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,"[Normal, Dry, Combination, Oily]"
3,3,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,"[Algae (Seaweed) Extract, Mineral Oil, Petrola...",Crème de la Mer Moisturizer,695.0,190.0,1,3.9,[Dry]
4,4,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,"[Aqua/Water/Eau, Glycerin, Propanediol, Dimeth...",The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.7,4.2,"[Normal, Dry]"


In [35]:
fdata.skin_type[0]

['Dry', 'Normal', 'Combination']

## One-Hot Encoding

The columns `skin_type` and `ingredients` contains a list of strings. We'll transform these columns by one-hot encoding them.

In [36]:
from sklearn.preprocessing import MultiLabelBinarizer

df = fdata.copy()
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('skin_type')),
                          columns=mlb.classes_,
                          index=df.index))

In [37]:
df.head()

,Unnamed: 0,Label,URL,brand,description,full_description,ingredients,name,num_reviews,price,prod_size,rating,Combination,Dry,Formulation,Normal,Oil,Oily,Sensitive,Serum
0,0,moisturizing-cream-oils-mists,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,"[Glacial Glycoprotein, Squalene, Aqua/Water, G...",Ultra Facial Cream,1132.0,32.0,1.7,4.3,1,1,0,1,0,0,0,0
1,1,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,"[Centella Asiatica Complex, Herbs Complex, Wat...",Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.7 fl.,4.1,1,1,0,1,0,1,0,0
2,2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,"[Signal Peptide Complex (Growth Factors), Pygm...",Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,1,1,0,1,0,1,0,0
3,3,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,"[Algae (Seaweed) Extract, Mineral Oil, Petrola...",Crème de la Mer Moisturizer,695.0,190.0,1,3.9,0,1,0,0,0,0,0,0
4,4,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,"[Aqua/Water/Eau, Glycerin, Propanediol, Dimeth...",The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.7,4.2,0,1,0,1,0,0,0,0


In our skin type column there were a few elements that do not refer to an actual skin type. Let's drop those.

In [39]:
df = df.drop(columns=['Formulation', 'Serum', 'Oil'])
df.columns

Index(['Unnamed: 0', 'Label', 'URL', 'brand', 'description',
       'full_description', 'ingredients', 'name', 'num_reviews', 'price',
       'prod_size', 'rating', 'Combination', 'Dry', 'Normal', 'Oily',
       'Sensitive'],
      dtype='object')

In [40]:
df.head()

,Unnamed: 0,Label,URL,brand,description,full_description,ingredients,name,num_reviews,price,prod_size,rating,Combination,Dry,Normal,Oily,Sensitive
0,0,moisturizing-cream-oils-mists,https://www.sephora.com/product/ultra-facial-c...,Kiehl's Since 1851,A bestselling face moisturizer that’s free of ...,NaN,"[Glacial Glycoprotein, Squalene, Aqua/Water, G...",Ultra Facial Cream,1132.0,32.0,1.7,4.3,1,1,1,0,0
1,1,moisturizing-cream-oils-mists,https://www.sephora.com/product/cicapair-tiger...,Dr. Jart+,A green-to-beige color-correcting treatment th...,NaN,"[Centella Asiatica Complex, Herbs Complex, Wat...",Cicapair™ Tiger Grass Color Correcting Treatme...,3065.0,52.0,1.7 fl.,4.1,1,1,1,1,0
2,2,moisturizing-cream-oils-mists,https://www.sephora.com/product/protini-tm-pol...,Drunk Elephant,A protein moisturizer that combines signal pep...,NaN,"[Signal Peptide Complex (Growth Factors), Pygm...",Protini™ Polypeptide Moisturizer,4872.0,68.0,1.69,4.0,1,1,1,1,0
3,3,moisturizing-cream-oils-mists,https://www.sephora.com/product/creme-de-la-me...,La Mer,A luxuriously rich cream that thoroughly sooth...,NaN,"[Algae (Seaweed) Extract, Mineral Oil, Petrola...",Crème de la Mer Moisturizer,695.0,190.0,1,3.9,0,1,0,0,0
4,4,moisturizing-cream-oils-mists,https://www.sephora.com/product/the-dewy-skin-...,Tatcha,A rich cream that feeds skin with plumping hyd...,NaN,"[Aqua/Water/Eau, Glycerin, Propanediol, Dimeth...",The Dewy Skin Cream Plumping & Hydrating Moist...,1727.0,68.0,1.7,4.2,0,1,1,0,0


In [41]:
ddf = df.copy()

mlb = MultiLabelBinarizer()
ddf = ddf.join(pd.DataFrame(mlb.fit_transform(ddf.pop('ingredients')),
                          columns=mlb.classes_,
                          index=ddf.index))

In [43]:
ddf.shape

(1196, 5251)

In [44]:
# merged = ddf.merge(df)
# merged.columns
# merged.to_csv('full_v5.csv')

In [47]:
def get_ingredient_count():
    import ast
    ingredient_count = {}  # maps ingredient: # used
    for ingredient_list in products_with_ingredients.ingredients:
        try:
            il = ast.literal_eval(ingredient_list)
#             il = json.loads(ingredient_list.replace("'", '"').replace('\\', ' '))
        except:
            continue
        parsed_ingredients = []
        for ing in il:
            # clean single ingredient
            ingredient = ing.strip(' .*[]')
            words = set(ingredient.split())
            if 'not' in words or 'are' in words or 'at' in words:
                continue
            
            if ingredient not in ingredient_count:
                ingredient_count[ingredient] = 1
            else:
                ingredient_count[ingredient] += 1
            parsed_ingredients.append(ingredient)
    return ingredient_count

def remove_all_paren(s, delim='()'):
    import re
    return re.sub(r'\((.*?)\)', '', s)

In [48]:
d = get_ingredient_count()

In [51]:
ddf.to_csv('sephora_one_hot_encoded_ingredients.csv')

In [293]:
l = []
for col in df.columns:
    try:
        l.append((col, sum(df[col])))
    except:
        continue

sorted(l, key=lambda x: x[1])[-100:]

[('Palmitoyl Tetrapeptide-7', 79),
 ('Adenosine', 80),
 ('Hydrogenated Polyisobutene', 80),
 ('Hydroxyethyl Acrylate/Sodium Acryloyldimethyl Taurate Copolymer', 85),
 ('Sodium Chloride', 87),
 ('Trehalose', 87),
 ('Sorbitan Isostearate', 90),
 ('Citral', 91),
 ('Citronellol', 95),
 ('Polysorbate 60', 97),
 ('Allantoin', 99),
 ('Hyaluronic Acid', 100),
 ('Lactic Acid', 108),
 ('Sodium Citrate', 108),
 ('Geraniol', 110),
 ('Hexylene Glycol', 111),
 ('Acrylates/C10-30 Alkyl Acrylate Crosspolymer', 112),
 ('Stearic Acid', 115),
 ('Fragrance', 117),
 ('Alcohol', 118),
 ('Cetyl Alcohol', 119),
 ('Niacinamide', 120),
 ('Butyrospermum Parkii (Shea) Butter', 123),
 ('Helianthus Annuus (Sunflower) Seed Oil', 126),
 ('Chlorphenesin', 127),
 ('Panthenol', 127),
 ('Polysorbate 20', 129),
 ('Camellia Sinensis Leaf Extract', 130),
 ('Disodium EDTA', 130),
 ('Lecithin', 131),
 ('Caffeine', 138),
 ('Disodium Edta', 139),
 ('Mica', 139),
 ('Silica', 149),
 ('Glyceryl Stearate', 155),
 ('2-Hexanediol', 1

In [63]:
ast.literal_eval("['Glacial Glycoprotein', 'Squalene', 'Aqua/Water', 'Glycerin', 'Cyclohexasiloxane', 'Squalane', 'Bis-PEG-18 Methyl Ether Dimethyl Silane', 'Sucrose Stearate', 'Stearyl Alcohol', 'PEG-8 Stearate', 'Myristyl Myristate', 'Pentaerythrityl Tetraethlhexanoate', 'Prunus Armeniaca Kernal Oil/Apricot Kernal Oil', 'Phenoxyethanol', 'Persea Gratissima Oil/Avocado Oil', 'Glyceryl Stearate', 'Cetl Alcohol', 'Oryza Sativa Bran Oil/Rice Bran Oil', 'Olea Europaea Fruit Oil/G4Olive Fruit Oil', 'Chlorphenesin', 'Stearic Acid', 'Palmitic Acid', 'Disodium EDTA', 'Acrylates/C10-30 Alkyl Acrylate Crosspolymer', 'Carbomer', 'Prunus Amygdalus Dulcis Oil/Sweet Almond Oil', 'Xanthan Gum', 'Ethylhexylglycerin', 'Sodium Hydroxide', 'Tocopherol', 'Glycine Soja Oil/Soybean Oil', 'Pseudoalteromonas Ferment Extract', 'Myristic acid', 'Hydroxypalmitol Sphinganine', 'BHT', 'Salicylic Acid', 'Citric Acid.']")

['Glacial Glycoprotein',
 'Squalene',
 'Aqua/Water',
 'Glycerin',
 'Cyclohexasiloxane',
 'Squalane',
 'Bis-PEG-18 Methyl Ether Dimethyl Silane',
 'Sucrose Stearate',
 'Stearyl Alcohol',
 'PEG-8 Stearate',
 'Myristyl Myristate',
 'Pentaerythrityl Tetraethlhexanoate',
 'Prunus Armeniaca Kernal Oil/Apricot Kernal Oil',
 'Phenoxyethanol',
 'Persea Gratissima Oil/Avocado Oil',
 'Glyceryl Stearate',
 'Cetl Alcohol',
 'Oryza Sativa Bran Oil/Rice Bran Oil',
 'Olea Europaea Fruit Oil/G4Olive Fruit Oil',
 'Chlorphenesin',
 'Stearic Acid',
 'Palmitic Acid',
 'Disodium EDTA',
 'Acrylates/C10-30 Alkyl Acrylate Crosspolymer',
 'Carbomer',
 'Prunus Amygdalus Dulcis Oil/Sweet Almond Oil',
 'Xanthan Gum',
 'Ethylhexylglycerin',
 'Sodium Hydroxide',
 'Tocopherol',
 'Glycine Soja Oil/Soybean Oil',
 'Pseudoalteromonas Ferment Extract',
 'Myristic acid',
 'Hydroxypalmitol Sphinganine',
 'BHT',
 'Salicylic Acid',
 'Citric Acid.']

In [75]:
# most 
ingredient_count = get_ingredient_count()
ingredient_count

Glacial Glycoprotein
Squalene
Aqua/Water
Glycerin
Cyclohexasiloxane
Squalane
Bis-PEG-18 Methyl Ether Dimethyl Silane
Sucrose Stearate
Stearyl Alcohol
PEG-8 Stearate
Myristyl Myristate
Pentaerythrityl Tetraethlhexanoate
Prunus Armeniaca Kernal Oil/Apricot Kernal Oil
Phenoxyethanol
Persea Gratissima Oil/Avocado Oil
Glyceryl Stearate
Cetl Alcohol
Oryza Sativa Bran Oil/Rice Bran Oil
Olea Europaea Fruit Oil/G4Olive Fruit Oil
Chlorphenesin
Stearic Acid
Palmitic Acid
Disodium EDTA
Acrylates/C10-30 Alkyl Acrylate Crosspolymer
Carbomer
Prunus Amygdalus Dulcis Oil/Sweet Almond Oil
Xanthan Gum
Ethylhexylglycerin
Sodium Hydroxide
Tocopherol
Glycine Soja Oil/Soybean Oil
Pseudoalteromonas Ferment Extract
Myristic acid
Hydroxypalmitol Sphinganine
BHT
Salicylic Acid
Citric Acid.
Centella Asiatica Complex
Herbs Complex
Minerals Solution and Panthenol
Water
Centella Asiatica Leaf Water
Isononyl Isononanoate
Caprylic / Capric Triglyceride
Isotridecyl Isononanoate
Butylene Glycol
Methyl Methacrylate Cross

**Althaea Officinalis Root Extract
Symphytum Officinale Leaf Extract
Trifolium Pratense (Clover) Flower Extract
Achillea Millefolium Flower Extract
Euphrasia Officinalis Extract
Thuja Occidentalis Leaf Extract
Menyanthes Trifoliata Leaf Extract
Salix Alba (Willow) Bark Extract
Chelidonium Majus Extract
Urea
Hydroxyethylpiperazine Ethane Sulfonic Acid
Glycosyl Trehalose
Hydrogenated Starch Hydrolysate
Acrylates/C10-30 Alkyl Acrylate Crosspolymer
Carbomer
Xanthan Gum
Trisodium EDTA
Tromethamine
***Frgrance
Citronnellol
Limonene.
*Napiers Original Formula.
**Napiers Moisture Formula.
***Fragrances of Natural Origin.
Clean at Sephora products are formulated without:
Sulfates—SLS + SLES
Parabens
Formaldehydes
Formaldehyde-releasing agents
Phthalates
Mineral Oil
Retinyl Palmitate
Oxybenzone
Coal Tar
Hydroquinone
Triclosan
Triclocarban
Undisclosed synthetic fragrances (Products can be formulated with disclosed synthetic fragrances that meet the following two criteria: (1) the synthetic fragra

styrene)
1
4 Dioxane in final formulas must comply with the thresholds as follows: (10 or < ppm for products that are meant to be rinsed off
wiped off or removed
3ppm or < for products that are meant to remain on the skin).
Water
Cyclopentasiloxane
Bis-Diglyceryl Polyacyladipate-2
Butylene Glycol
Ppg-2 Myristyl Ether Propionate
Cetyl Alcohol
Peg-40 Stearate
Butyrospermum Parkii (Shea Butter)
Squalane
Glyceryl Stearate
Sorbitan Stearate
Epilobium Angustifolium Extract
Yeast Extract/Faex/Extrait De Levure
Beta-Carotene
Sodium Hyaluronate
Citrus Grandis (Grapefruit) Peel Oil
Tocopheryl Acetate
Methyl Glucose Sesquistearate
Pelargonium Graveolens (Geranium) Flower Oil
Dimethicone Crosspolymer
Acrylates/C10-30 Alkyl Acrylate Crosspolymer
Carbomer
Peg-20 Methyl Glucose Sesquistearate
C12-15 Alkyl Ethylhexanoate
Tetrasodium Edta
Sodium Hydroxide
Panthenol
Magnesium Ascorbyl Phosphate
Limonene
Citronellol
Geraniol
Linalool
Chlorphenesin
Phenoxyethanol
Hyaluronic Acid
Rosehip Oil, Camellia Oil,

Carbomer
Propanediol
Sodium Chloride
Sodium Glycolate
Citric Acid
Sodium Hydroxide
Sodium Benzoate
Potassium Sorbate
Phenoxyethanol.
Clean at Sephora products are formulated without:Sulfates—SLS + SLES
Parabens
Formaldehydes
Formaldehyde-releasing agents
Phthalates
Mineral Oil
Retinyl Palmitate
Oxybenzone
Coal Tar
Hydroquinone
Triclosan
Triclocarban
Undisclosed synthetic fragrances (Products can be formulated with disclosed synthetic fragrances that meet the following two criteria: (1) the synthetic fragrances do not include any of the ingredients listed in numbers 1 through 12 above and (2) the synthetic fragrances are at a concentration below 1% of the total formula) The following type of acrylates: (ethyl acrylate
ethyl methacrylate
methyl methacrylate
butyl methacrylate
hydroxypropyl methacrylate
tetrahydrofurfuryl methacrylate
trimethylolpropane trimethacrylate
aluminum salts)
Animal Oils/Musks/Fats
Benzophenone + Related Compounds
Butoxyethanol
Carbon Black
Lead/Lead Acetate
Meth

Sclerotium Gum
Caprylyl Glycol
Polysorbate 60
Phenoxyethanol
Potassium Sorbate
Fragrance/Parfum
Silica
Titanium Dioxide (CI 77891)
Iron Oxides (CI 77491)
Sodium Benzoate
Sodium Metabisulfite
Benzyl Benzoate
Limonene.
Meadowfoam Seed Oil
Vitamin E
Rosewater
Chamomile Wax
Limnanthes Alba (Meadowfoam) Seed Oil
Water
Cera Alba (Beeswax)
Pentylene Glycol
Prunus Amygdalus Dulcis (Sweet Almond) Oil
Citrus Limon (Lemon) Peel Oil
Osmanthus Fragrans Flower Extract
Rosa Damascena Flower Water
Anthemis Nobilis Flower Oil
Ormenis Multicaulis Flower Wax
Tocopherol
Sodium Carbonate
Limonene
Citronellol
Geraniol
Citral.
Clean at Sephora products are formulated without:
Sulfates—SLS + SLES
Parabens
Formaldehydes
Formaldehyde-releasing agents
Phthalates
Mineral Oil
Retinyl Palmitate
Oxybenzone
Coal Tar
Hydroquinone
Triclosan
Triclocarban
Undisclosed synthetic fragrances (Products can be formulated with disclosed synthetic fragrances that meet the following two criteria: (1) the synthetic fragrances do n

...

3ppm or < for products that are meant to remain on the skin).
Vitamin E
Rice Bran
Rosemary
Caprylic/Capric Triglycerides
Prunus Armeniaca (Apricot) Kernel Oil
PEG-40 Sorbitan Peroleate
Helianthus Annuus (Sunflower) Seed Oil
Citrus Aurantium Dulcis (Orange) Oil
Citrus Grandis (Grapefruit) Peel Oil
Cymbopogon Schoenanthus Oil
Borago Officinalis Seed Oil
Lavandula Angustifolia (Lavender) Oil
Rosmarinus Officinalis (Rosemary) Leaf Extract
Solanum Lycopersicum (Tomato) Extract
Lavandula Hybrida Oil
Citrus Nobilis (Mandarin Orange) Peel Oil
Oryza Sativa (Rice) Bran Oil
Carthamus Tinctorius (Safflower) Seed Oil
Aleurites Moluccana Seed Oil
Cetyl Ethylhexanoate
Dicaprylyl Carbonate
Tocopheryl Acetate
Tocopherol
C12-15 Alkyl Benzoate
Citral
Limonene
Linalool.
Natural Salicylic Acid
Organic Essential Oil Complex and Grape Seed Polyphenols
Grape Water
Water
Vitis Vinifera (Grape) Fruit Water
Decyl Glucoside
Glycerin
Sodium Cocoyl Glutamate
Sucrose Cocoate
Acacia Senegal Gum
Xanthan Gum
Propanedio

Vegan Collagen
Water (Aqua
Eau)
Caprylic/Capric Triglyceride
Hydrogenated Polyisobutene
Glycerin
Chlorella Protothecoides Oil
Dimethicone
Butylene Glycol
Glyceryl Stearate
PEG-100 Stearate
Butyrospermum Parkii (Shea) Butter
Pentylene Glycol
Potassium Cetyl Phosphate
Parachlorella Beijerinckii Exopolysaccharides
Collagen (Vegan)*
Ceramide NP
Cynara Scolymus (Artichoke) Leaf Extract
Retinyl Palmitate
Astragalus Membranaceus Root Extract
Atractylodes Macrocephala Root Extract
Iris Florentina Root Extract
Bupleurum Falcatum Root Extract
Copper Lysinate/Prolinate
Methylglucoside Phosphate
Stearalkonium Bentonite
Acrylates/C10-30 Alkyl Acrylate Crosspolymer
Silica
Caprylyl Glycol
Sodium Hydroxide
Disodium EDTA
Ethylhexylglycerin
Cetyl Alcohol
Propylene Carbonate
Quartz
Phenoxyethanol
Leuconostoc/Radish Root Ferment Filtrate
Chlorphenesin
Fragrance (Parfum)
Coumarin.
Grape Water
Instant Foaming Cleanser:
Aqua/Water/Eau
Glycerin*
Sodium Cocoyl Glutamate*
Caprylyl/Capryl Glucoside*
Cocamidoprop

Sodium Sulfite
Sodium Metabisulfite
Steareth-20
Dipeptide-2
Ethylhexylglycerin
Carbomer
Tromethamine
Sorbic Acid
Chlorphenesin
Phenoxyethanol
Mica
Titanium Dioxide (Ci 77891)
Iron Oxides (Ci 77491
Ci 77492
Ci 77499).
Clean at Sephora products are formulated without:
Sulfates—SLS + SLES
Parabens
Formaldehydes
Formaldehyde-releasing agents
Phthalates
Mineral Oil
Retinyl Palmitate
Oxybenzone
Coal Tar
Hydroquinone
Triclosan
Triclocarban
Undisclosed synthetic fragrances (Products can be formulated with disclosed synthetic fragrances that meet the following two criteria: (1) the synthetic fragrances do not include any of the ingredients listed in numbers 1 through 12 above and (2) the synthetic fragrances are at a concentration below 1% of the total formula) The following type of acrylates: (ethyl acrylate
ethyl methacrylate
methyl methacrylate
butyl methacrylate
hydroxypropyl methacrylate
tetrahydrofurfuryl methacrylate
trimethylolpropane trimethacrylate
aluminum salts)
Animal Oils/Musks/Fa

Toluene
Butylated hydroxyanisole (BHA)
Butylated hydroxytoluene (BHT) that is 0.1% or more of total formula
Ethanolamines DEA/TEA/MEA/ETA
Nanoparticles as defined by the European Commission
Petrolatum and Parrafin that is not USP grade
Phenoxyethanol that is 1% or more of total formulation
Polyacrylamide & Acrylamide
The following types of Styrene (Bromostyrene
Deastyrene/acrylates/dvbcopolymer
sodium styrene/divinylbenzene copolymer
styrene oxide
styrene)
1
4 Dioxane in final formulas must comply with the thresholds as follows: (10 or < ppm for products that are meant to be rinsed off
wiped off or removed
3ppm or < for products that are meant to remain on the skin).
Alguronic Acid
Plant-Derived Collagen
Algae Prebiotic
Gentle Rejuvenating Cleanser:
Water (Aqua
Eau)
Sodium Cocoyl Isethionate
Glyceryl Stearate SE
Cocamidopropyl Betaine
Coconut Acid
Cetyl Alcohol
Palmitic Acid
Stearic Acid
Sodium Isethionate
Dimethicone PEG-8 Meadowfoamate
Parachlorella Beijerinckii Exopolysaccharides
Gl

{'': 2,
 'Wild Indigo Seed': 1,
 'Propylene Glycol Dimethicone Glycerin Ascorbic Acid Ethylhexyl Palmitate Cetyl Peg/Ppg-10/1 Dimethicone Dimethicone Crosspolymer Ascorbyl Glucoside Cyclohexasiloxane Hydroxyethylpiperazine Ethane Sulfonic Acid Lauroyl Lysine Acrylonitrile/Methyl Methacrylate/Vinylidene Chloride Copolymer Polysilicone-11 Adenosine Hydrolyzed Hyaluronic Acid Limonene Isobutane Citrus Aurantium Dulcis (Orange) Peel Oil Citrus Limon Peel Oil / Lemon Peel Oil Citral Pentaerythrityl Tetra-Di-T-Butyl Hydroxyhydrocinnamate.': 1,
 'Diethylhexyl Succinate': 1,
 'Yeast and Plankton Extracts': 1,
 'Vitamin ': 2,
 'Natural Fragrance': 8,
 '*Avena Sativa (Oat) Kernel Extract': 1,
 'Cermide EOP*. *One of the five Ceramides of the 5-Cera Complex.': 1,
 'Certified Organic Cornflower and Camomile': 1,
 'Fragaria Ananassa (Strawberry) Fruit Extract': 1,
 "Beeswax(Cera Alba/Cire D'abeille)": 1,
 'Fragrance/Parfum.': 9,
 'Hydrolyzed Corn Starch Octenylsuccinate': 1,
 'Red 27': 1,
 'Coffea 

## Cleaning Prod Size

When parsing with selenium, we did not fully clean the product size. We'll make sure these are floats and all in fl oz.

In [52]:
ddf.prod_size.head()

0        1.7
1    1.7 fl.
2       1.69
3          1
4        1.7
Name: prod_size, dtype: object

In [ ]:
data = ddf.copy()
for i in data.index:
    prod_size = data.prod_size[i]
    url = data.URL[i]
    
    try:
        prod_size = float(prod_size)
    except ValueError:
        num_pattern = r'([\d\.]+)'
        match = re.search(num_pattern, prod_size)
        if match:
            prod_size = match.group(1)
        else:
            prod_size = 0
    data.prod_size[i] = prod_size
    data.URL[i] = url

In [56]:
data.prod_size.head()

0     1.7
1     1.7
2    1.69
3       1
4     1.7
Name: prod_size, dtype: object

In [ ]:
cosm = pd.read_csv('sephora_one_hot_encoded_ingredients.csv')

In [ ]:
# some extra cleaning and processing of urls.. moved to data_scape

description_counts = cosm.groupby('description').count()
description_counts.loc[description_counts['URL'] > 2]

def corrected_urls(names_with_incorrect_urls):
    chrome_path = '/Users/marissakhoury/Desktop/webscrape/drivers/chromedriver'
    # initiate empty dataframe
    df = pd.DataFrame(columns=['prod_name', 'URL'])    
    for name in names_with_incorrect_urls:
        driver = webdriver.Chrome(executable_path = chrome_path)
        driver.get('https://www.sephora.com')
        time.sleep(3)
        driver.find_element_by_xpath('/html/body/div[1]/div[1]/header/div[1]/div[1]/div/div[1]/div/form/div/div/div/div/input').send_keys(name, Keys.ENTER)
        time.sleep(2)
        try:
            x_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div/div/button')
            ActionChains(driver).click(x_button).perform()
            time.sleep(2)
        except NoSuchElementException:
            pass
        time.sleep(1)
        element_results = driver.find_elements_by_class_name('css-ix8km1')
        if not element_results:
            # assume it went directly to the page
            corrected_url = driver.current_url
        else:
            corrected_url = element_results[0].get_attribute('href') # url for this product, assume it's the first result
#         for a in element_results:
#             subURL = a.get_attribute('href')
#             subpageURL.append(subURL)
    # transform into a data frame
        driver.close()
        dic = {'prod_name': name, 'URL': corrected_url}
        print(dic)
        df = df.append(dic, ignore_index = True)
    print(df)
    # add columns
    df2 = pd.DataFrame(columns=['brand', 'name', 'price', 'rank', 'skin_type', 'ingredients'])
    df = pd.concat([df, df2], axis = 1)
    
    return df
corrected_df = corrected_urls(names_with_incorrect_urls)

# Testing this on Jet Lag Mask :) 

chrome_path = '/Users/marissakhoury/Desktop/webscrape/drivers/chromedriver'
driver = webdriver.Chrome(executable_path = chrome_path)
url = 'https://www.sephora.com'
driver.get(url)
time.sleep(1)

# initiate empty dataframe
df_jet_lag = pd.DataFrame(columns=['prod_name', 'URL'])
driver.find_element_by_xpath('/html/body/div[1]/div[1]/header/div[1]/div[1]/div/div[1]/div/form/div/div/div/div/input').send_keys('Jet Lag Mask', Keys.ENTER)
time.sleep(1)
try:
    x_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[2]/div/div/button')
    ActionChains(driver).click(x_button).perform()
    time.sleep(2)
except NoSuchElementException:
        pass
# do something like this...
element = driver.find_elements_by_class_name('css-ix8km1')
print(element)

subpageURL = []

for a in element:
    subURL = a.get_attribute('href')
    subpageURL.append(subURL)
    print(subURL)
    find_name = driver.find_elements_by_class_name('css-pelz90')
    prod_name_to_map.append(find_name)
    dic = {'prod_name': names_with_incorrect_urls, 'URL': subpageURL}
    df_jet_lag = df_jet_lag.append(pd.DataFrame(dic), ignore_index = True)
print(df_jet_lag)
data = pd.read_csv('corrected_labels.csv')
data.tail()

In [23]:
data = pd.read_csv('data_without_labels')

## Cleaning the Labels for our products

Previously labels were set to the tickers we used to find the product on Sephora's site. However, these are not readable fields and not the best groupings for skin care products. Below, I wrote a function that visits the product's url and parsed the product label sephora classified it as.

In [ ]:

epath = '/Users/marissakhoury/Library/Application Support/Google/Chrome/Default/Extensions/cjpalhdlnbpafiamejdnhcphjbkeiagm/1.31.2_0'
chrome_path = '/Users/marissakhoury/Desktop/webscrape/drivers/chromedriver'

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

chrome_options = Options()
chrome_options.add_argument('load-extension=' + epath)
chrome_options.add_argument('chrome')

def get_label(url):
    driver = webdriver.Chrome(chrome_path, chrome_options=chrome_options)
    driver.create_options()
    driver.get(url)
    try:
        cl = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/main/div/div[1]/nav/ol/li[3]/a').text
        driver.close()
        return cl
    except Exception as e:
        print(e)
        driver.close()
        return None

cosm = pd.read_csv('sephora_one_hot_encoded_ingredients.csv')
for i in cosm[cosm.Label.isna()].index:
    url = cosm.URL[i]
    label = get_label(url)
    cosm.Label[i] = label 
    

In [12]:
cosm.to_csv('data_clean_revised_labels.csv')

In [13]:
cosm.Label.isna().sum()

61

In [ ]:
dd = pd.read_csv('TEMP_1_4_2021')

In [ ]:
full_data = pd.read_csv('corrected_labels.csv')
full_data.Label.isna().sum()

In [8]:
cosm.to_csv('final_data_clean.csv')

In [ ]:
for i in cosm[cosm.Label.isna()].index:
    url = cosm.URL[i]
    print(i)
    label = get_label(url)
    cosm.Label[i] = label 